In [1]:
from sagemaker.predictor import retrieve_default
endpoint_name = "jumpstart-dft-llama-codellama-13b-i-20240417-060043"
predictor = retrieve_default(endpoint_name)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [57]:
from utils import get_answer,followup_prompt 

In [17]:
# from https://github.com/mykter/afl-training/tree/main/harness
libh = '''
#include <unistd.h>
// an 'nprintf' implementation - print the first len bytes of data
void lib_echo(char *data, ssize_t len);

// optimised multiply - returns x*y
int  lib_mul(int x, int y);
'''
libc = '''
#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <assert.h>

#include "library.h"

void lib_echo(char *data, ssize_t len){
	if(strlen(data) == 0) {
		return;
	}
	char *buf = calloc(1, len);
	strncpy(buf, data, len);
	printf("%s",buf);
	free(buf);

	// A crash so we can tell the harness is working for lib_echo
	if(data[0] == 'p') {
		if(data[1] == 'o') {
			if(data[2] =='p') {
				if(data[3] == '!') {
					assert(0);
				}
			}
		}
	}
}

int  lib_mul(int x, int y){
	if(x%2 == 0) {
		return y << x;
	} else if (y%2 == 0) {
		return x << y;
	} else if (x == 0) {
		return 0;
	} else if (y == 0) {
		return 0;
	} else {
		return x * y;
	}
}
'''

In [20]:
function_name="lib_echo"

role = f"You are a developer. you need to write a libFuzzer target for the {function_name} function of your c library."


template= '''
#include <stdint.h>
#include <stdlib.h>
'''+'''
// Step 2 include custom and standard headers
'''+'''
<FILL>

extern "C" int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
'''+f'''
// Step 3 call to the {function_name} function
<FILL>;
''' +'''
  return 0;
}
'''
graph='}'
prompt = f"""
{role}

Work step by step:
1. Identify and analyze {function_name} in the library source code: which headers and parameters does it need to be properly executed?
2. Fill the template with necessary headers
3. fill the template LLVMFuzzerTestOneInput function with a call to {function_name}, such that {function_name} can be properly fuzz
Answer should be code only, no explanation.

## Library:
library.c
```c
{libc}
```
library.h
```c
{libh}
```
## End of library

## template:
```c
{template}
```
## End of template:
 """
prompt = '[INST]' + prompt + '[/INST]'

In [3]:
payload = {
    "inputs": "import socket\n\ndef ping_exponential_backoff(host: str):",
    "parameters": {
        "max_new_tokens": 256,
        "top_p": 0.9,
        "temperature": 0.2
    }
}
response = predictor.predict(payload)
print(response)

[{'generated_text': '\n    """\n    Ping a host with exponential backoff.\n\n    :param host: The host to ping.\n    :return: True if the host is reachable, False otherwise.\n    """\n    backoff_time = 1\n    while True:\n        try:\n            socket.create_connection((host, 80), timeout=1).close()\n            return True\n        except OSError:\n            pass\n        backoff_time *= 2\n        if backoff_time > 10:\n            backoff_time = 10\n        print(f"Ping failed, retrying in {backoff_time} seconds...")\n        time.sleep(backoff_time)\n'}]


In [5]:
print(response[0]['generated_text'])


    """
    Ping a host with exponential backoff.

    :param host: The host to ping.
    :return: True if the host is reachable, False otherwise.
    """
    backoff_time = 1
    while True:
        try:
            socket.create_connection((host, 80), timeout=1).close()
            return True
        except OSError:
            pass
        backoff_time *= 2
        if backoff_time > 10:
            backoff_time = 10
        print(f"Ping failed, retrying in {backoff_time} seconds...")
        time.sleep(backoff_time)



In [23]:
r = get_answer(prompt)

========= PAYLOAD =========
payload = {
    "inputs": "[INST]\nYou are a developer. you need to write a libFuzzer target for the lib_echo function of your c library.\n\nWork step by step:\n1. Identify and analyze lib_echo in the library source code: which headers and parameters does it need to be properly executed?\n2. Fill the template with necessary headers\n3. fill the template LLVMFuzzerTestOneInput function with a call to lib_echo, such that lib_echo can be properly fuzz\nAnswer should be code only, no explanation.\n\n## Library:\nlibrary.c\n```c\n\n#include <stdlib.h>\n#include <stdio.h>\n#include <string.h>\n#include <assert.h>\n\n#include \"library.h\"\n\nvoid lib_echo(char *data, ssize_t len){\n\tif(strlen(data) == 0) {\n\t\treturn;\n\t}\n\tchar *buf = calloc(1, len);\n\tstrncpy(buf, data, len);\n\tprintf(\"%s\",buf);\n\tfree(buf);\n\n\t// A crash so we can tell the harness is working for lib_echo\n\tif(data[0] == 'p') {\n\t\tif(data[1] == 'o') {\n\t\t\tif(data[2] =='p') {\n\t

In [26]:
print(r[0]['generated_text'])

  ```
#include <stdint.h>
#include <stdlib.h>
#include <string.h>
#include "library.h"

extern "C" int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
    char *data = (char *)Data;
    lib_echo(data, Size);
    return 0;
}
```


In [41]:
lines = r[0]['generated_text'].split('\n')
start_idx = next(i for i, line in enumerate(lines) if '```' in line) + 1
end_idx = next(i for i, line in enumerate(lines) if i > start_idx and '```' in line)
c_code = '\n'.join(lines[start_idx:end_idx])
with open('file.c', 'w') as file:
    file.write(c_code)


In [53]:
import subprocess

compile_command = ['afl-gcc-fast', '-fsanitize=fuzzer', 'file.c', '-o', 'file']
result = subprocess.run(compile_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if result.returncode == 0:
    print("Compilation successful.")
else:
    print("Compilation failed.")
    print("Errors:", result.stderr)
    with open('error.log', 'w') as file:
        file.write(result.stderr)

Compilation failed.
Errors: /usr/bin/ld: /tmp/ccKVPC42.o: in function `LLVMFuzzerTestOneInput':
/home/sagemaker-user/DemoNotebooks/file.c:8: undefined reference to `lib_echo'
collect2: error: ld returned 1 exit status



In [44]:
with open('error.log', 'r') as file:
    content = file.read()
fix_compilation_prompt = followup_prompt(prompt,  r[0]['generated_text'], "can you regenerate the previous code to fix the following error:\n" + content)

In [48]:
r = get_answer(fix_compilation_prompt)

========= PAYLOAD =========
payload = {
    "inputs": "<s>[INST]\nYou are a developer. you need to write a libFuzzer target for the lib_echo function of your c library.\n\nWork step by step:\n1. Identify and analyze lib_echo in the library source code: which headers and parameters does it need to be properly executed?\n2. Fill the template with necessary headers\n3. fill the template LLVMFuzzerTestOneInput function with a call to lib_echo, such that lib_echo can be properly fuzz\nAnswer should be code only, no explanation.\n\n## Library:\nlibrary.c\n```c\n\n#include <stdlib.h>\n#include <stdio.h>\n#include <string.h>\n#include <assert.h>\n\n#include \"library.h\"\n\nvoid lib_echo(char *data, ssize_t len){\n\tif(strlen(data) == 0) {\n\t\treturn;\n\t}\n\tchar *buf = calloc(1, len);\n\tstrncpy(buf, data, len);\n\tprintf(\"%s\",buf);\n\tfree(buf);\n\n\t// A crash so we can tell the harness is working for lib_echo\n\tif(data[0] == 'p') {\n\t\tif(data[1] == 'o') {\n\t\t\tif(data[2] =='p') {\

In [46]:
fix_compilation_prompt

'<s>[INST]\nYou are a developer. you need to write a libFuzzer target for the lib_echo function of your c library.\n\nWork step by step:\n1. Identify and analyze lib_echo in the library source code: which headers and parameters does it need to be properly executed?\n2. Fill the template with necessary headers\n3. fill the template LLVMFuzzerTestOneInput function with a call to lib_echo, such that lib_echo can be properly fuzz\nAnswer should be code only, no explanation.\n\n## Library:\nlibrary.c\n```c\n\n#include <stdlib.h>\n#include <stdio.h>\n#include <string.h>\n#include <assert.h>\n\n#include "library.h"\n\nvoid lib_echo(char *data, ssize_t len){\n\tif(strlen(data) == 0) {\n\t\treturn;\n\t}\n\tchar *buf = calloc(1, len);\n\tstrncpy(buf, data, len);\n\tprintf("%s",buf);\n\tfree(buf);\n\n\t// A crash so we can tell the harness is working for lib_echo\n\tif(data[0] == \'p\') {\n\t\tif(data[1] == \'o\') {\n\t\t\tif(data[2] ==\'p\') {\n\t\t\t\tif(data[3] == \'!\') {\n\t\t\t\t\tassert(0)

In [ ]:
def add prompt